In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

<H2> Функция для запроса данных с бесконечно пролистываемой страницы

In [2]:
def get_Comics_List(page: int):
    json ={
        "sort":"rate",
        "dir":"desc",
        "page":page,
        "types":["9", "8"],
        "site_id":"1",
        "type":"manga",
        "caution_list":["Отсутствует", "16+", "18+"]
    } # json для полечения комиксов западных и русских
    headers ={
        "X-Csrf-Token":None,
        "X-Requested-With":"XMLHttpRequest",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0"
    }
    url = "https://mangalib.me/api/list"

    session = requests.session()

    html = session.get("https://mangalib.me") #Получаем базовую страницу
    soup = BeautifulSoup(html.text, 'lxml')
    csrf_token = soup.find("meta", {"name":"_token"}).get("content")
    headers["X-Csrf-Token"] = csrf_token #Запоминаем токен

    session.headers.update(headers)
    manga_list = session.post(url, json=json) #Получаем данные
    return manga_list.json()

<h4> Считываем первые 1200 записей

In [3]:
manga_mas = []
for i in range(20):
    manga_mas.append(get_Comics_List(i+1))
manga_mas

[{'items': {'current_page': 1,
   'data': [{'id': 8135,
     'name': 'Love Advice from the Great Duke of Hell',
     'eng_name': '',
     'rus_name': 'Любовный совет от великого герцога из Ада',
     'cover': '9wQk93wUnvzr',
     'slug': 'love-advice-from-the-great-duke-of-hell',
     'type_id': 9,
     'created_at': '2018-12-14T05:32:54.000000Z',
     'updated_at': '2023-10-23T03:41:02.000000Z',
     'site': 1,
     'type': 'Комикс западный',
     'modelType': 'manga',
     'covers': {'default': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_250x350.jpg',
      'thumbnail': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_thumb.jpg'},
     'coverImage': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk93wUnvzr_250x350.jpg',
     'coverImageThumbnail': 'https://cover.imglib.info/uploads/cover/love-advice-from-the-great-duke-of-hell/cover/9wQk

Смотрим данные, которые мы вытянули из запросов

In [4]:
manga_mas[0]['items']['data'][0].keys()

dict_keys(['id', 'name', 'eng_name', 'rus_name', 'cover', 'slug', 'type_id', 'created_at', 'updated_at', 'site', 'type', 'modelType', 'covers', 'coverImage', 'coverImageThumbnail', 'href', 'title_status_id'])

<h2> Заполняем DF

In [5]:
from random import random
from time import sleep
href_mas = []
data_mas = []
df = pd.DataFrame()
i = 0
for manga_mas_lower in manga_mas:
    data = manga_mas_lower['items']['data']
    
    for elems in data:
        for key in list(elems.keys()):
            # print(elems[key])
            if key != 'covers':
                df.loc[i, key] = elems[key]
            else:
                for kkey in elems[key].keys():
                    df.loc[i, "cover_"+kkey] = elems[key][kkey]
        # df.loc[i,'href'] = elems['href']
        # df.loc[i,'id'] = int(elems['id'])
        i+=1
df.head()


,id,name,eng_name,rus_name,cover,slug,type_id,created_at,updated_at,site,type,modelType,cover_default,cover_thumbnail,coverImage,coverImageThumbnail,href,title_status_id
0,8135.0,Love Advice from the Great Duke of Hell,,Любовный совет от великого герцога из Ада,9wQk93wUnvzr,love-advice-from-the-great-duke-of-hell,9.0,2018-12-14T05:32:54.000000Z,2023-10-23T03:41:02.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://cover.imglib.info/uploads/cover/love-a...,https://mangalib.me/love-advice-from-the-great...,NaN
1,19009.0,The Little Trashmaid,The Little Trashmaid,Маленькая мусорная русалочка,KvRCkR9vnCfL,the-little-trashmaid,9.0,2019-11-30T09:31:44.000000Z,2023-10-23T03:45:06.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://cover.imglib.info/uploads/cover/the-li...,https://mangalib.me/the-little-trashmaid,NaN
2,45864.0,Тёмный замок,The Dark Castle,Тёмный Замок,8tyXnxq0GvGc,temnyy-zamok,8.0,2020-08-30T09:02:21.000000Z,2023-10-23T03:44:41.000000Z,1.0,Руманга,manga,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://cover.imglib.info/uploads/cover/temnyy...,https://mangalib.me/temnyy-zamok,NaN
3,9545.0,Brutus &amp; Pixie,Brutus And Pixie,Брут и Пикси,sMeSbOWbTEOi,brutus-and-pixie,9.0,2019-02-11T15:28:42.000000Z,2023-10-23T03:45:03.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://cover.imglib.info/uploads/cover/brutus...,https://mangalib.me/brutus-and-pixie,NaN
4,8288.0,Stalker x Stalker,Stalker x Stalker,Сталкер и Сталкер,jpmkjMlmmyxD,stalker-x-stalker,9.0,2018-12-23T05:53:32.000000Z,2023-10-23T03:41:53.000000Z,1.0,Комикс западный,manga,https://cover.imglib.info/uploads/cover/stalke...,https://cover.imglib.info/uploads/cover/stalke...,https://cover.imglib.info/uploads/cover/stalke...,https://cover.imglib.info/uploads/cover/stalke...,https://mangalib.me/stalker-x-stalker,NaN


Функция для красивого статус бара

In [6]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )


<h2>Читаем данные со страниц комиксов

In [7]:
import sys

aim_heads = {'Год релиза':'Release_Year', 'Статус тайтла':'Title_Status', 'Статус перевода':'Translate_Status', 
             'Автор':'Author', 'Художник':'Artist',  'Возрастной рейтинг':'Age_restriction', 'Загружено глав':'Chapters_Loaded'}
for href in enumerate(log_progress(df['href'], every = 1)):
    heads = []
    bodies = {}
    response = requests.get(href[1])
    bs = BeautifulSoup(response.text)
    # Колонка с базовыми данными о комиксе
    heads_bs = bs.find_all('div', {"class":"media-info-list__title"})
    bodies_bs = bs.find_all('div', {"class":"media-info-list__value"})
    
    for i in heads_bs:
        heads.append(i.text)
    for i in enumerate(heads):
        if i[1] in aim_heads.keys() :
            # print(aim_heads[i[1]])
            bodies[aim_heads[i[1]]] = bodies_bs[i[0]].text.replace('\n','') #''.join(filter(str.isalnum, bodies_bs[i[0]].text))
    # print(heads)
    # print(bodies)
    for i in bodies.keys():
        df.loc[href[0], i] = bodies[i]

    # Краткое описание
    desc = bs.find('div', {'class':'media-description__text'})
    if desc:
        df.loc[href[0], 'Short_Description'] = desc.text.replace('\n',' ').replace('\r',' ')
        # print(df.loc[href[0], 'Short_Description'])
    
    # Множество тэгов
    tags = bs.find('div', {'class':'media-tags'})
    tag_mas = []
    if tags != None:
        for i in tags:
            txt = i.text.replace('\n','')
            if txt != '':
                tag_mas.append(txt)
    df.loc[href[0], 'tags'] = str(tag_mas)
    
    # Списки пользователей и оценки
    user_lists = {}
    user_rates = {}
    media_cols = bs.find_all('div', {'class':'media-section__col'})
    for col in media_cols:
        col_type = col.find('div', {'class':'media-section__title'}).text
        if col_type != "Оценки пользователей":
            divs = col.find_all('div',{'class':'media-stats-item__column media-stats-item__count'})
            names = col.find_all('div',{'class':'media-stats-item__column media-stats-item__title'})
            for i in range(len(divs)):
                user_lists[names[i].text.replace('\n','')] = int(divs[i].text)
        else:
            divs = col.find_all('div',{'class':'media-stats-item__column media-stats-item__count'})
            names = col.find_all('div',{'class':'media-stats-item__column media-stats-item__title'})
            for i in range(len(divs)):
                user_rates[int(names[i].text.replace('\n',''))] = int(divs[i].text)
    df.loc[href[0], 'User_rates'] = str(user_rates)
    df.loc[href[0], 'User_lists'] = str(user_lists)

    # Списки переводчиков
    trans_dict = {}
    trans = bs.find('div', {'class':'team-list'})
    if trans != None:
        tr_hrefs = trans.find_all('a', {'class':'team-list-item team-list-item_xs'})
        tr_names = trans.find_all('div', {'class':'team-list-item__name'})
        for i in range(len(tr_names)):
            trans_dict[tr_names[i].text] = tr_hrefs[i].get('href')
    df.loc[href[0], 'Translators'] = str(trans_dict)
df.head()


,id,name,eng_name,rus_name,cover,slug,type_id,created_at,updated_at,site,...,Translate_Status,Author,Artist,Age_restriction,Chapters_Loaded,Short_Description,tags,User_rates,User_lists,Translators
0,8135.0,Love Advice from the Great Duke of Hell,,Любовный совет от великого герцога из Ада,9wQk93wUnvzr,love-advice-from-the-great-duke-of-hell,9.0,2018-12-14T05:32:54.000000Z,2023-10-23T03:41:02.000000Z,1.0,...,Завершен,Unfins,Unfins,16+,138,Девушка твоей мечты едва ли знает о т...,"['комедия', 'повседневность', 'романтика', 'св...","{10: 14791, 9: 180, 8: 345, 7: 33, 6: 95, 5: 9...","{'Читаю': 47987, 'В планах': 51715, 'Брошено':...",{'Noah Studio': 'https://mangalib.me/team/noah...
1,19009.0,The Little Trashmaid,The Little Trashmaid,Маленькая мусорная русалочка,KvRCkR9vnCfL,the-little-trashmaid,9.0,2019-11-30T09:31:44.000000Z,2023-10-23T03:45:06.000000Z,1.0,...,Продолжается,s0s2,s0s2,NaN,135,Комикс о жизни современной русалочки....,"['арт', 'драма', 'комедия', 'повседневность', ...","{10: 7840, 9: 87, 8: 305, 7: 34, 6: 80, 5: 7, ...","{'Читаю': 70538, 'В планах': 22964, 'Брошено':...",{'Flying Island Team 2.0': 'https://mangalib.m...
2,45864.0,Тёмный замок,The Dark Castle,Тёмный Замок,8tyXnxq0GvGc,temnyy-zamok,8.0,2020-08-30T09:02:21.000000Z,2023-10-23T03:44:41.000000Z,1.0,...,Продолжается,faun_me,faun_me,16+,37,"Тёмный, мрачный замок, с бесконечными...","['дзёсэй', 'драма', 'комедия', 'мистика', 'ром...","{10: 6815, 9: 61, 8: 122, 7: 11, 6: 34, 5: 5, ...","{'Читаю': 38203, 'В планах': 27005, 'Брошено':...",{}
3,9545.0,Brutus &amp; Pixie,Brutus And Pixie,Брут и Пикси,sMeSbOWbTEOi,brutus-and-pixie,9.0,2019-02-11T15:28:42.000000Z,2023-10-23T03:45:03.000000Z,1.0,...,Продолжается,pet_foolery,pet_foolery,NaN,334,Комикс про необычную дружбу милого и ...,"['героическое фэнтези', 'комедия', 'повседневн...","{10: 5808, 9: 40, 8: 258, 7: 20, 6: 61, 5: 2, ...","{'Читаю': 40246, 'В планах': 27764, 'Брошено':...","{'SHADOW': 'https://mangalib.me/team/shadow', ..."
4,8288.0,Stalker x Stalker,Stalker x Stalker,Сталкер и Сталкер,jpmkjMlmmyxD,stalker-x-stalker,9.0,2018-12-23T05:53:32.000000Z,2023-10-23T03:41:53.000000Z,1.0,...,Заморожен,Merryweatherey,Princess Hinghoi,16+,97,"Юнко и Юкио — яндере и сталкер, влюбл...","['комедия', 'повседневность', 'романтика', 'тр...","{10: 3647, 9: 36, 8: 790, 7: 37, 6: 500, 5: 22...","{'Читаю': 51137, 'В планах': 33054, 'Брошено':...",{'PantsuTeam': 'https://mangalib.me/team/pants...


"Потрачено на открытие страницы:  0.6658093929290771  
Потрачено на заполнение df:  0.02171039581298828"  
Судя по всему, больше всего времени тратится на открытие каждой новой страницы

Описание итогового DF

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 1199
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1200 non-null   float64
 1   name                 1200 non-null   object 
 2   eng_name             1200 non-null   object 
 3   rus_name             1200 non-null   object 
 4   cover                1200 non-null   object 
 5   slug                 1200 non-null   object 
 6   type_id              1200 non-null   float64
 7   created_at           1200 non-null   object 
 8   updated_at           1200 non-null   object 
 9   site                 1200 non-null   float64
 10  type                 1200 non-null   object 
 11  modelType            1200 non-null   object 
 12  cover_default        1200 non-null   object 
 13  cover_thumbnail      1200 non-null   object 
 14  coverImage           1200 non-null   object 
 15  coverImageThumbnail  1200 non-null   object

title_status_id - бесполезный параметр, его следует убрать

In [9]:
df.drop('title_status_id', axis=1)
df.head()

,id,name,eng_name,rus_name,cover,slug,type_id,created_at,updated_at,site,...,Translate_Status,Author,Artist,Age_restriction,Chapters_Loaded,Short_Description,tags,User_rates,User_lists,Translators
0,8135.0,Love Advice from the Great Duke of Hell,,Любовный совет от великого герцога из Ада,9wQk93wUnvzr,love-advice-from-the-great-duke-of-hell,9.0,2018-12-14T05:32:54.000000Z,2023-10-23T03:41:02.000000Z,1.0,...,Завершен,Unfins,Unfins,16+,138,Девушка твоей мечты едва ли знает о т...,"['комедия', 'повседневность', 'романтика', 'св...","{10: 14791, 9: 180, 8: 345, 7: 33, 6: 95, 5: 9...","{'Читаю': 47987, 'В планах': 51715, 'Брошено':...",{'Noah Studio': 'https://mangalib.me/team/noah...
1,19009.0,The Little Trashmaid,The Little Trashmaid,Маленькая мусорная русалочка,KvRCkR9vnCfL,the-little-trashmaid,9.0,2019-11-30T09:31:44.000000Z,2023-10-23T03:45:06.000000Z,1.0,...,Продолжается,s0s2,s0s2,NaN,135,Комикс о жизни современной русалочки....,"['арт', 'драма', 'комедия', 'повседневность', ...","{10: 7840, 9: 87, 8: 305, 7: 34, 6: 80, 5: 7, ...","{'Читаю': 70538, 'В планах': 22964, 'Брошено':...",{'Flying Island Team 2.0': 'https://mangalib.m...
2,45864.0,Тёмный замок,The Dark Castle,Тёмный Замок,8tyXnxq0GvGc,temnyy-zamok,8.0,2020-08-30T09:02:21.000000Z,2023-10-23T03:44:41.000000Z,1.0,...,Продолжается,faun_me,faun_me,16+,37,"Тёмный, мрачный замок, с бесконечными...","['дзёсэй', 'драма', 'комедия', 'мистика', 'ром...","{10: 6815, 9: 61, 8: 122, 7: 11, 6: 34, 5: 5, ...","{'Читаю': 38203, 'В планах': 27005, 'Брошено':...",{}
3,9545.0,Brutus &amp; Pixie,Brutus And Pixie,Брут и Пикси,sMeSbOWbTEOi,brutus-and-pixie,9.0,2019-02-11T15:28:42.000000Z,2023-10-23T03:45:03.000000Z,1.0,...,Продолжается,pet_foolery,pet_foolery,NaN,334,Комикс про необычную дружбу милого и ...,"['героическое фэнтези', 'комедия', 'повседневн...","{10: 5808, 9: 40, 8: 258, 7: 20, 6: 61, 5: 2, ...","{'Читаю': 40246, 'В планах': 27764, 'Брошено':...","{'SHADOW': 'https://mangalib.me/team/shadow', ..."
4,8288.0,Stalker x Stalker,Stalker x Stalker,Сталкер и Сталкер,jpmkjMlmmyxD,stalker-x-stalker,9.0,2018-12-23T05:53:32.000000Z,2023-10-23T03:41:53.000000Z,1.0,...,Заморожен,Merryweatherey,Princess Hinghoi,16+,97,"Юнко и Юкио — яндере и сталкер, влюбл...","['комедия', 'повседневность', 'романтика', 'тр...","{10: 3647, 9: 36, 8: 790, 7: 37, 6: 500, 5: 22...","{'Читаю': 51137, 'В планах': 33054, 'Брошено':...",{'PantsuTeam': 'https://mangalib.me/team/pants...


Итого: 28 различных параметров на 1200 объектов
В ходе выполнения данной лабораторной работы я познакомился с библиотекой BeautifulSoup и научился добывать данные из веб-страниц  
Полученные данные в дальнейшем могут быть использованы для комплексного сравнения различных комиксов и их жанров по популярности